In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
import time

In [ ]:
import ipytest
from context import Context
from tools.notebooks import env
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from utils.logger import get_logger

logger = get_logger("manual_interactor")
ipytest.autoconfig()

context = Context()

In [ ]:
from contracts.staking_contract import StakingContract, EnterFarmEvent, ExitFarmEvent, ClaimRewardsFarmEvent
from utils.contract_data_fetchers import StakingContractDataFetcher
staking_contract: StakingContract
staking_contract = context.get_contracts(config.STAKINGS_V2)[0]
print(f"Using: {staking_contract.address} farm token: {staking_contract.farm_token}")

In [ ]:
from trackers.staking_economics_tracking import StakingEconomics

staking_tracker = StakingEconomics(staking_contract.address, context.network_provider)

In [ ]:
from contracts.metastaking_contract import MetaStakingContract, EnterMetastakeEvent, ExitMetastakeEvent, ClaimRewardsMetastakeEvent
from utils.contract_data_fetchers import MetaStakingContractDataFetcher
metastaking_contract: MetaStakingContract
metastaking_contract = context.get_contracts(config.METASTAKINGS_BOOSTED)[0]

Gather users with tokens

In [ ]:
from utils.utils_scenarios import collect_farm_contract_users
from multiversx_sdk import ApiNetworkProvider
from utils.utils_scenarios import FetchedUser
from typing import List

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
fetched_users = collect_farm_contract_users(100, staking_contract.address, staking_contract.farming_token, staking_contract.farm_token,
                                            mainnet_api, context.network_provider.proxy)

users: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
if not users:
    raise Exception('No users found for the given criteria')

In [29]:
users: List[FetchedUser] = fetched_users.get_users_with_both_tokens()
if not users:
    raise Exception('No users found for the given criteria')

Set user from gathered data

In [ ]:
from utils.decoding_structures import METASTAKE_TOKEN_ATTRIBUTES, FARM_TOKEN_ATTRIBUTES, STAKE_V2_TOKEN_ATTRIBUTES, STAKE_V1_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, get_all_token_nonces_details_for_account, Account, WrapperAddress

index = 0
shard = 1
user = Account(pem_file=config.DEFAULT_ACCOUNTS)
if shard >= 0:
    i = 0
    for u in users:
        if u.address.get_shard() == shard:
            if i == index:
                user.address = u.address
                user.sync_nonce(context.network_provider.proxy)
                print(f'User {i}: {user.address.bech32()} shard: {user.address.get_shard()}')
                break
            i += 1
    if i != index:
        raise Exception(f'User {index} not found for shard {shard}')
else:
    user.address = users[index].address
    user.sync_nonce(context.network_provider.proxy)
    print(f'User {index}: {user.address.bech32()} shard: {user.address.get_shard()}')

tokens_in_account = get_all_token_nonces_details_for_account(staking_contract.farm_token, user.address.bech32(), context.network_provider.proxy)

print(user.address.bech32())
print(f'Stake Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    try:
        stake_token_decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), STAKE_V2_TOKEN_ATTRIBUTES)
    except ValueError as e:
        # handle for old stake token attributes
        stake_token_decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), STAKE_V1_TOKEN_ATTRIBUTES)
    print(f'Stake Tokens: {stake_token_decoded_attributes}')

In [ ]:
from utils.decoding_structures import METASTAKE_TOKEN_ATTRIBUTES, FARM_TOKEN_ATTRIBUTES, STAKE_V2_TOKEN_ATTRIBUTES, STAKE_V1_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, get_all_token_nonces_details_for_account, Account, WrapperAddress

index = -1
shard = 1
user = Account(pem_file=config.DEFAULT_ACCOUNTS)
filtered_users = [u for u in users if u.address.get_shard() == shard]
user = filtered_users[index]

tokens_in_account = get_all_token_nonces_details_for_account(staking_contract.farm_token, user.address.bech32(), context.network_provider.proxy)

print(user.address.bech32())
print(f'Stake Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    try:
        stake_token_decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), STAKE_V2_TOKEN_ATTRIBUTES)
    except ValueError as e:
        # handle for old stake token attributes
        stake_token_decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), STAKE_V1_TOKEN_ATTRIBUTES)
    print(f'Stake Tokens: {stake_token_decoded_attributes}')

Find user by token

In [ ]:
from multiversx_sdk.network_providers import ApiNetworkProvider
from utils.utils_chain import WrapperAddress, dec_to_padded_hex, get_all_token_nonces_details_for_account, decode_merged_attributes, base64_to_hex
from utils.decoding_structures import STAKE_V2_TOKEN_ATTRIBUTES, STAKE_V1_TOKEN_ATTRIBUTES

migration_nonce = 729837

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
for nonce in reversed(range(migration_nonce)):
    print(f'Current nonce: {nonce}')
    url = f'nfts/{staking_contract.farm_token}-{dec_to_padded_hex(nonce)}/accounts'
    try:
        response = mainnet_api.do_get_generic(url)
    except Exception as e:
        continue

    print(response)
    user_found = ""
    for entry in response:
        if WrapperAddress(entry['address']).is_smart_contract():
            continue
        user_found = entry['address']
    
    print(f'Potential user: {user_found} for nonce {dec_to_padded_hex(nonce)}')

    if user_found:
        tokens_in_account = get_all_token_nonces_details_for_account(staking_contract.farm_token, user_found, context.network_provider.proxy)
        for token in tokens_in_account:
            print(token["attributes"])
            try:
                stake_token_decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), STAKE_V2_TOKEN_ATTRIBUTES)
            except ValueError as e:
                try:
                    # handle for old stake token attributes
                    stake_token_decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), STAKE_V1_TOKEN_ATTRIBUTES)
                except ValueError as e:
                    # unstake token
                    continue
            print(f'Stake Tokens nonce {dec_to_padded_hex(nonce)}: {stake_token_decoded_attributes}')

Set user from given bech32 address

In [ ]:
from utils.decoding_structures import METASTAKE_TOKEN_ATTRIBUTES, FARM_TOKEN_ATTRIBUTES, STAKE_V2_TOKEN_ATTRIBUTES, STAKE_V1_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, get_all_token_nonces_details_for_account, Account, WrapperAddress

user = Account(pem_file="~/Documents/sh1.pem")
user.address = WrapperAddress("erd1wwx5zhmx9mag9k8zuajj8c9zce6mzerfr3m9ck6l7c00dwkq2a2ssyfnzk")
user.sync_nonce(context.network_provider.proxy)
tokens_in_account = get_all_token_nonces_details_for_account(staking_contract.farm_token, user.address.bech32(), context.network_provider.proxy)

print(f'Stake Tokens in account:')
print(tokens_in_account)
for token in tokens_in_account:
    print(f'\t{token}')
    try:
        stake_token_decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), STAKE_V2_TOKEN_ATTRIBUTES)
    except ValueError as e:
        # handle for old stake token attributes
        stake_token_decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), STAKE_V1_TOKEN_ATTRIBUTES)
    print(f'Stake Tokens: {stake_token_decoded_attributes}')

In [ ]:
def get_stats_for_user(staking_contract: StakingContract, user: Account):
    energy_contract: SimpleLockEnergyContract
    energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
    user_energy = energy_contract.get_energy_for_user(context.network_provider.proxy, user.address.bech32())
    logger.debug(f'User energy on energy factory: {user_energy}')
    
    logger.debug(f"Stats for user: {user.address.bech32()} on staking {staking_contract.address}")
    staking_stats = staking_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
    staking_stats.update(staking_contract.get_all_stats(context.network_provider.proxy))
    logger.debug(f"Staking stats: {staking_stats}")

get_stats_for_user(staking_contract, user)

In [ ]:
ride_purse = Address('')
depositer = Account(pem_file=env.USER1_PEM)
depositer.address = ride_purse
depositer.sync_nonce(context.network_provider.proxy)

upgrade

In [ ]:
from contracts.staking_contract import StakingContractVersion
# upgrade all farms
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[StakingContract] = context.get_contracts(config.STAKINGS_V2)
for contract in contracts:
    contract.version = StakingContractVersion.V2
    tx_hash = contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         config.STAKING_V3_BYTECODE_PATH, [], no_init=True)
    context.network_provider.check_complex_tx_status(tx_hash, "staking upgrade")

contract config

In [ ]:
from time import sleep
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[StakingContract] = context.get_contracts(config.STAKINGS_V2)
simple_lock_energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
for contract in contracts:
    tx_hash = contract.set_boosted_yields_rewards_percentage(context.deployer_account, context.network_provider.proxy, 6000)
    tx_hash = contract.set_boosted_yields_factors(context.deployer_account, context.network_provider.proxy, 
                                                  [2, 1, 0, 1, 1])
    tx_hash = contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)
    tx_hash = simple_lock_energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, contract.address)
    sleep(2)

In [ ]:
# resume all farms
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[StakingContract] = context.get_contracts(config.STAKINGS_V2)
for contract in contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

transfer tokens and prepare accounts

In [ ]:
staking_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 1000000000)

In [ ]:
staking_contract.add_admin(context.deployer_account, context.network_provider.proxy, depositer.address.bech32())
staking_contract.topup_rewards(depositer, context.network_provider.proxy, 10000000000000000000)

In [ ]:
staking_contract.set_max_apr(context.deployer_account, context.network_provider.proxy, 10000)

In [ ]:
staking_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)

In [ ]:
txhash = staking_contract.register_farm_token(context.deployer_account, context.network_provider.proxy, ["STKTST", "STKTST"])
status = context.network_provider.check_simple_tx_status(txhash, "set register farm token")

In [ ]:
status = context.network_provider.check_simple_tx_status("5f5e3b111bc5ca2f59e4f8bb8faf3aa7e7827bc0247808b53eb7bd58e9db19e4", "set boosted yields factors in farm")

In [ ]:
farm_token_hex = MetaStakingContractDataFetcher(Address(metastaking_contract.address), context.network_provider.proxy.url).get_data("getDualYieldTokenId")
print(hex_to_string(farm_token_hex))

enter stake

In [ ]:
event = EnterFarmEvent(staking_contract.farming_token, 0, 1000000000000000000, "", 0, 0, False, False)
txhash = staking_contract.stake_farm(context.network_provider, depositer, event, True)

claim stake

In [ ]:
txhash = staking_contract.allow_external_claim(context.network_provider, user)

In [ ]:
event = ClaimRewardsFarmEvent(100000000000000000, 10, "")
txhash = staking_contract.claim_rewards(context.network_provider, depositer, event)

In [ ]:
from utils.utils_chain import get_token_details_for_address

farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(staking_contract.farm_token, user.address.bech32(), context.network_provider.proxy)
event = ClaimRewardsFarmEvent(farm_token_amount, farm_token_nonce, "")
# get_stats_for_user(staking_contract, user)
txhash = staking_contract.claim_rewards(context.network_provider, user, event)
time.sleep(6 if user.address.get_shard() == 1 else 40)
get_stats_for_user(staking_contract, user)

claim boosted rewards

In [ ]:
from utils.utils_chain import get_token_details_for_address

farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(staking_contract.farm_token, user.address.bech32(), context.network_provider.proxy)
event = ClaimRewardsFarmEvent(farm_token_amount, farm_token_nonce, "")
# get_stats_for_user(staking_contract, user)
txhash = staking_contract.claim_boosted_rewards(context.network_provider, user, event)
time.sleep(6 if user.address.get_shard() == 1 else 40)
get_stats_for_user(staking_contract, user)

exit stake

In [ ]:
event = ExitFarmEvent(staking_contract.farm_token, 1000000, 9, "", 1000000)
txhash = staking_contract.unstake_farm(context.network_provider, depositer, event)

In [ ]:
from utils.utils_chain import get_token_details_for_address

farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(staking_contract.farm_token, depositer.address.bech32(), context.network_provider)
event = ClaimRewardsFarmEvent(farm_token_amount, farm_token_nonce, "")
txhash = staking_contract.claim_rewards(context.network_provider, depositer, event)

report contract data

In [ ]:
staking_tracker.update_data()
staking_tracker.report_current_tracking_data()

Shadowfork control

In [ ]:
from contracts.builtin_contracts import SFControlContract

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
sf_control_contract.epoch_fast_forward(context.deployer_account, context.network_provider.proxy, 16, 20)

In [ ]:
from multiversx_sdk import CodeMetadata
from utils.utils_tx import upgrade_call

dummy_sc = "erd1qqqqqqqqqqqqqpgqwjpvqpv36ujpaqmj7kh8flvwaqy0mjqhvmusma3y0w"
owner = Account(pem_file=config.DEFAULT_ADMIN)
owner.address = Address("erd1rzv9c5wps2e78lpdq6pf9qkx5wlkr2yceuynmsd98hm3gtp8vmuse6y69m")
owner.sync_nonce(context.network_provider.proxy)

metadata = CodeMetadata(upgradeable=True, payable_by_contract=False, readable=True)
upgrade_call("dummy", context.network_provider.proxy, 100000000, owner, Address(dummy_sc), 
             Path().home() / "projects" / "contracts" / "mx-sc-dummy-proxy" / "output-docker" / "dummy-proxy" / "dummy-proxy.wasm", metadata, [])

In [ ]:
from utils.utils_tx import endpoint_call

endpoint_call(context.network_provider.proxy, 100000000, owner, Address(dummy_sc), "callInternalTransferEndpoint", 
              ['STADA-833615', 14172, 27512970695365523642357, 'erd1qqqqqqqqqqqqqpgqnyq8k8nfurx5rz7zxudfeeqm983uw2tvkp2shvf2ls', 'unstakeFarm'])

In [ ]:
from utils.utils_chain import nominated_amount
staking_contract.topup_rewards(context.deployer_account, context.network_provider.proxy, nominated_amount(500000))

In [ ]:
staking_contract.resume(context.deployer_account, context.network_provider.proxy)

In [ ]:
staking_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)